# Importing Libraries

In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.__version__

'2.4.1'

# Load the dataset

In [30]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [31]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [32]:
y = df.iloc[:, -1].values # take the last column 
X = df.iloc[:, 3:-1].values # take from 3 column upto last column

In [33]:
X.shape

(10000, 10)

# Data preprocessing

## Encode categorical data

In [34]:
# Label encode the gender column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# our gender column is the 3rd column
X[:, 2] = le.fit_transform(X[:,2]) 

In [35]:
# One hot encode geography column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], 
                       remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [36]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

# Splitting the dataset into the training and testing set

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Feature Scaling

In [38]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Building Neural Network

## Initialize ANN

In [39]:
ann_model = tf.keras.models.Sequential()

## Adding the first hidden layer

In [40]:
ann_model.add(tf.keras.layers.Dense(6, activation='relu'))

# Adding second hidden layer

In [41]:
ann_model.add(tf.keras.layers.Dense(6, activation='relu'))

# Adding output layer

In [42]:
ann_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  

# Training the ANN model

## Compile the model

In [43]:
ann_model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

## Fit(train) the model

In [44]:
ann_model.fit(X_train, y_train,
              batch_size=32, 
              epochs=25)

Epoch 1/25
235/235 [==============================] - 1s 1ms/step - loss: 0.5513 - accuracy: 0.7906
Epoch 2/25
235/235 [==============================] - 0s 1ms/step - loss: 0.4859 - accuracy: 0.7933
Epoch 3/25
235/235 [==============================] - 0s 1ms/step - loss: 0.4600 - accuracy: 0.7962
Epoch 4/25
235/235 [==============================] - 0s 1ms/step - loss: 0.4362 - accuracy: 0.7989
Epoch 5/25
235/235 [==============================] - 0s 1ms/step - loss: 0.4404 - accuracy: 0.7921
Epoch 6/25
235/235 [==============================] - 0s 1ms/step - loss: 0.4267 - accuracy: 0.7991
Epoch 7/25
235/235 [==============================] - 0s 1ms/step - loss: 0.4284 - accuracy: 0.8047
Epoch 8/25
235/235 [==============================] - 0s 1ms/step - loss: 0.4288 - accuracy: 0.8015
Epoch 9/25
235/235 [==============================] - 0s 1ms/step - loss: 0.4226 - accuracy: 0.8076
Epoch 10/25
235/235 [==============================] - 0s 1ms/step - loss: 0.4320 - accuracy: 0.8083

# Make Predictions and evaluating the model

# Making prediction on a single datapoint

Geography: France  
Credit Score: 600  
Gender: Male  
Age: 40 yrs  
Tenure: 3yrs   
Balance: $ 60000  

Number of Products: 2  
Has credit card: yes  
Is active: yes  
Estimated salary: $50000 

In [54]:
test1 = [600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 5000]
test1[2] = le.transform([test1[2]])
test = np.array(ct.transform([test1]))
test

array([[1.0, 0.0, 0.0, 600, array([1]), 40, 3, 60000, 2, 1, 1, 5000]],
      dtype=object)

In [55]:
# Apply the same transformation 
test = sc.transform(test)

In [57]:
test

array([[ 0.98465111, -0.56811207, -0.57468161, -0.51937478,  0.91974271,
         0.11067641, -0.68335613, -0.24998937,  0.80424154,  0.64365658,
         0.96668786, -1.6552634 ]])

In [58]:
ann_model.predict(test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0]], dtype=int32)

predict method will return the probaility value because we have uses sigmoind activation in the final layer

In [59]:
ann_model.predict_classes(test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0]], dtype=int32)

In [62]:
np.argmax(ann_model.predict(test), axis=-1)

array([0])

# Evaluating the model

In [64]:
_, acc = ann_model.evaluate(X_test, y_test)
print(acc)

79/79 [==============================] - 0s 2ms/step - loss: 0.3521 - accuracy: 0.8576
0.8575999736785889
